In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataprep.eda import plot, plot_missing, plot_correlation, create_report

import pretty_midi
import os
import joblib

from sklearn.preprocessing import MinMaxScaler

In [63]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [2]:
midi_file = "bach342.mid"
midi_file = pretty_midi.PrettyMIDI(midi_file)

In [3]:
midi_file.instruments

[Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1"),
 Instrument(program=6, is_drum=False, name="Track 1")]

In [4]:
midi_file2 = "handel107.mid"
midi_file2 = pretty_midi.PrettyMIDI(midi_file2)

In [5]:
midi_file2.instruments

[Instrument(program=70, is_drum=False, name="Joshua"),
 Instrument(program=40, is_drum=False, name="Strings I"),
 Instrument(program=48, is_drum=False, name="Strings II"),
 Instrument(program=41, is_drum=False, name="Viola"),
 Instrument(program=42, is_drum=False, name="Cello"),
 Instrument(program=43, is_drum=False, name="Double Bass"),
 Instrument(program=6, is_drum=False, name="Harpsichord Treble"),
 Instrument(program=6, is_drum=False, name="Harpsichord Bass")]

In [6]:
midi_file3 = "handel147.mid"
midi_file3 = pretty_midi.PrettyMIDI(midi_file3)

C:\Users\yousi\anaconda3\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [7]:
midi_file3.instruments

[Instrument(program=68, is_drum=False, name="Oboe I"),
 Instrument(program=68, is_drum=False, name="Oboe II"),
 Instrument(program=70, is_drum=False, name="Bassoon"),
 Instrument(program=48, is_drum=False, name="Violin I"),
 Instrument(program=48, is_drum=False, name="Violin II"),
 Instrument(program=41, is_drum=False, name="Viola"),
 Instrument(program=42, is_drum=False, name="Cello"),
 Instrument(program=43, is_drum=False, name="Double Bass"),
 Instrument(program=6, is_drum=False, name="Harpsichord RH"),
 Instrument(program=6, is_drum=False, name="Harpsichord LH")]

In [8]:
midi_file3.instruments[0].notes

[Note(start=0.000000, end=1.379309, pitch=64, velocity=75),
 Note(start=1.551723, end=2.058188, pitch=72, velocity=75),
 Note(start=2.068964, end=3.448273, pitch=71, velocity=75),
 Note(start=3.620687, end=4.127152, pitch=69, velocity=75),
 Note(start=4.137928, end=5.517237, pitch=75, velocity=75),
 Note(start=5.689651, end=5.937496, pitch=73, velocity=75),
 Note(start=5.948272, end=6.196116, pitch=71, velocity=75),
 Note(start=6.206892, end=7.586201, pitch=79, velocity=75),
 Note(start=7.758615, end=8.265080, pitch=79, velocity=75),
 Note(start=8.275856, end=9.655165, pitch=79, velocity=75),
 Note(start=9.827579, end=10.334044, pitch=78, velocity=75),
 Note(start=10.344820, end=11.724129, pitch=78, velocity=75),
 Note(start=11.896543, end=12.403008, pitch=76, velocity=75),
 Note(start=12.413784, end=13.793093, pitch=75, velocity=75),
 Note(start=13.965507, end=14.471972, pitch=78, velocity=75),
 Note(start=14.482748, end=15.862057, pitch=79, velocity=75),
 Note(start=16.034471, end=16

In [10]:
midi_file1 = midi_file
midi_file1.get_tempo_changes()

(array([0.]), array([120.]))

In [11]:
def extract_features(midi_file: str):
    """
    This function will extract features from the midi files and extracts their features
        Parameters:
            midi_file: a string that has the file path to the midi file
        returns:
            ([notes], [durations]): a list with notes and their duration
    
    """
    MAX_NUM_OF_POINTS = 4000
    STRIDE = 5 
    
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    notes = [0] * MAX_NUM_OF_POINTS
    durations = [0] * MAX_NUM_OF_POINTS
    # this doesn't take into account the instrument name. We took this approach because
    # different composers have different instrument name and it's not unified
    # thus, it will introduce a level of complexity and it might not be all that beneficial
    # if we have too many different instruments which is what we've observed from a quick check
    
    i = 0
    max_reached = False
    
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            if i == MAX_NUM_OF_POINTS:
                max_reached = True
                break
            notes[i] = note.pitch
            durations[i] = note.end - note.start
            i += 1
        if max_reached:
            break
        
    # to keep the data smaller, we'll only use 800 points i.e. every 5th (STRIDE) note and duration
    # of the first 4000 (MAX_NUM_OF_POINTS) values. If the number of notes exceed 4000, the rest will
    # be truncated. If we have less than MAX_NUM_OF_POINTS points, we will resort to padding with zeros in the end.
    # TODO: check if pre-padding is better
    # TODO: check if picking the last 4000 points is better for the model
    
    return notes[::STRIDE], durations[::STRIDE] 

In [17]:
dataset_directory ="data/train"

composers = [] # this will hold the composers' names

features_list = []
labels_list = [] # composer name corresponding to features in features_list

for composer in os.listdir(dataset_directory):
    if composer.startswith("."): # to avoid processing ".DS_Store"
        continue

    composers.append(composer)
    composer_directory = os.path.join(dataset_directory, composer)
    for midi_file in os.listdir(composer_directory):
        if midi_file.startswith("."): # to avoid processing ".DS_Store"
            continue
            
        file_path = os.path.join(composer_directory, midi_file)

        notes, durations = extract_features(file_path)
        features_list.append([notes, durations])
        labels_list.append(composer)

C:\Users\yousi\anaconda3\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


The code above takes a while to run, thus we will persist the data using **joblib**

In [79]:
features_list_tensor = tf.constant(features_list, dtype=tf.float32)
joblib.dump(features_list_tensor, "features_list_tensor.joblib")
joblib.dump(labels_list, "labels_list.joblib")

['labels_list.joblib']

In [82]:
features_list_tensor = joblib.load("features_list_tensor.joblib")
labels_list = joblib.load("labels_list.joblib")